In [1]:
import argparse
import json
from torch.utils.data import DataLoader
from thop import profile
from ptflops import get_model_complexity_info
import math

In [2]:
from models import *
from utils.datasets import *
from utils.utils import *
from quant_dorefa import *

from mymodel import *
from torchsummary import summary

In [3]:
model = torch.load('weights/4w4a/test_best.pt', map_location=torch.device('cpu'))
# model = UltraNet()
# model.load_state_dict(checkpoint[])
# model.hyp = hyp
# model.nc = 1
# model.arc = 'default'

In [4]:
# print(model.parameters())
# print(model['model'])
# for layers in model:
#     print(layers)
# summary(model, (3, 320, 160))

In [ ]:
print("Model's state_dict:")
for param_tensor in model['model']:
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())
#     if(len(model['model'][param_tensor].size()) > 1):
    print(param_tensor, model['model'][param_tensor].size())

# print(model['model']['layers.0.weight'])    

In [ ]:
conv_0_weight = model['model']['layers.0.weight']
conv_1_weight = model['model']['layers.4.weight']
conv_2_weight = model['model']['layers.8.weight']
conv_3_weight = model['model']['layers.12.weight']
conv_4_weight = model['model']['layers.16.weight']
conv_5_weight = model['model']['layers.19.weight']
conv_6_weight = model['model']['layers.22.weight']
conv_7_weight = model['model']['layers.25.weight']
conv_8_weight = model['model']['layers.28.weight']
print(conv_0_weight)
# torch.save(conv_0_weight, 'conv_0_original.txt')

In [6]:
bn_1_weight = model['model']['layers.1.weight']
bn_5_weight = model['model']['layers.5.weight']
bn_9_weight = model['model']['layers.9.weight']
bn_13_weight = model['model']['layers.13.weight']
bn_17_weight = model['model']['layers.17.weight']
bn_20_weight = model['model']['layers.20.weight']
bn_23_weight = model['model']['layers.23.weight']
bn_26_weight = model['model']['layers.26.weight']

bn_1_bias = model['model']['layers.1.bias']
bn_5_bias = model['model']['layers.5.bias']                           
bn_9_bias = model['model']['layers.9.bias']                          
bn_13_bias = model['model']['layers.13.bias']                                       
bn_17_bias = model['model']['layers.17.bias']                          
bn_20_bias = model['model']['layers.20.bias']                          
bn_23_bias = model['model']['layers.23.bias']                          
bn_26_bias = model['model']['layers.26.bias']     
# print(bn_1_weight)
print("Max and Min value in bn_1_weight:", bn_1_weight.max(), bn_1_weight.min())
print("Max and Min value in bn_5_weight:", bn_5_weight.max(), bn_5_weight.min())
print("Max and Min value in bn_9_weight:", bn_9_weight.max(), bn_9_weight.min())
print("Max and Min value in bn_13_weight:", bn_13_weight.max(), bn_13_weight.min())
print("Max and Min value in bn_17_weight:", bn_17_weight.max(), bn_17_weight.min())
print("Max and Min value in bn_20_weight:", bn_20_weight.max(), bn_20_weight.min())
print("Max and Min value in bn_23_weight:", bn_23_weight.max(), bn_23_weight.min())
print("Max and Min value in bn_26_weight:", bn_26_weight.max(), bn_26_weight.min())
print("Max and Min value in bn_1_bias:", bn_1_bias.max(), bn_1_bias.min())

Max and Min value in bn_1_weight: tensor(0.74779) tensor(0.24101)
Max and Min value in bn_5_weight: tensor(0.61837) tensor(0.19282)
Max and Min value in bn_9_weight: tensor(0.71140) tensor(0.16660)
Max and Min value in bn_13_weight: tensor(0.48525) tensor(0.19018)
Max and Min value in bn_17_weight: tensor(1.01425) tensor(0.19093)
Max and Min value in bn_20_weight: tensor(0.66609) tensor(0.20678)
Max and Min value in bn_23_weight: tensor(1.26842) tensor(0.22393)
Max and Min value in bn_26_weight: tensor(1.41240) tensor(0.13694)
Max and Min value in bn_1_bias: tensor(4.11842) tensor(-3.47873)


In [7]:
def get_w_quant(in_f, k):
    in_f = torch.tanh(in_f)
    n = float(2 ** (k-1)  - 1)
    in_f = in_f / torch.max(torch.abs(in_f))
    out = torch.round(in_f * n) / n
    return out

def get_w_int(in_f, k):
    in_f = torch.tanh(in_f)
    n = float(2 ** (k-1)  - 1)
    in_f = in_f / torch.max(torch.abs(in_f))
    out = torch.round(in_f * n)
    return out

def get_wbnI(in_f, K_bit, W_bit, Si, So):
    wbnI = (2**W_bit/(2**W_bit-1))*Si*in_f*2**K_bit/So
    return wbnI

def get_wbn_int(in_f, K_bit, W_bit, Si, So):
    wbnI = (2**W_bit/(2**W_bit-1))*Si*in_f*2**K_bit/So
    wbn_int = torch.round(wbnI)
    return wbn_int

def power_log(x):
    return 2**(math.ceil(math.log(x, 2)))

def get_bbnI(in_f, K_bit, W_bit, Si, So):
    bbnI = (in_f*2**(W_bit-1+K_bit)/So)*2**K_bit
    return bbnI

def get_bbn_int(in_f, K_bit, W_bit, Si, So):
    bbnI = (in_f*2**(W_bit-1+K_bit)/So)*2**K_bit
    bbn_int = torch.round(bbnI)
    return bbn_int

In [ ]:
bn_0_I = get_wbnI(bn_1_weight, 12, 4, 1, 1)
bn_1_I = get_wbnI(bn_5_weight, 12, 4, 1, 1)
bn_2_I = get_wbnI(bn_9_weight, 12, 4, 1, 1)
bn_3_I = get_wbnI(bn_13_weight, 12, 4, 1, 1)
bn_4_I = get_wbnI(bn_17_weight, 12, 4, 1, 1)
bn_5_I = get_wbnI(bn_20_weight, 12, 4, 1, 1)
bn_6_I = get_wbnI(bn_23_weight, 12, 4, 1, 1)
bn_7_I = get_wbnI(bn_26_weight, 12, 4, 1, 1)
print("bn_1_weight range:", bn_0_I.max().item(), bn_0_I.min().item(), "Required bitwidth: ", math.log(power_log(bn_0_I.max().data), 2))
print("bn_5_weight range:", bn_1_I.max().item(), bn_1_I.min().item(), "Required bitwidth: ", math.log(power_log(bn_1_I.max().data), 2))
print("bn_9_weight range:", bn_2_I.max().item(), bn_2_I.min().item(), "Required bitwidth: ", math.log(power_log(bn_2_I.max().data), 2))
print("bn_13_weight range:", bn_3_I.max().item(), bn_3_I.min().item(), "Required bitwidth: ", math.log(power_log(bn_3_I.max().data), 2))
print("bn_17_weight range:", bn_4_I.max().item(), bn_4_I.min().item(), "Required bitwidth: ", math.log(power_log(bn_4_I.max().data), 2))
print("bn_20_weight range:", bn_5_I.max().item(), bn_5_I.min().item(), "Required bitwidth: ", math.log(power_log(bn_5_I.max().data), 2))
print("bn_23_weight range:", bn_6_I.max().item(), bn_6_I.min().item(), "Required bitwidth: ", math.log(power_log(bn_6_I.max().data), 2))
print("bn_26_weight range:", bn_7_I.max().item(), bn_7_I.min().item(), "Required bitwidth: ", math.log(power_log(bn_7_I.max().data), 2))

In [20]:
bn_0_w_int = get_wbn_int(bn_1_weight, 12, 4, 1, 1)
bn_1_w_int = get_wbn_int(bn_5_weight, 12, 4, 1, 1)
bn_2_w_int = get_wbn_int(bn_9_weight, 12, 4, 1, 1)
bn_3_w_int = get_wbn_int(bn_13_weight, 12, 4, 1, 1)
bn_4_w_int = get_wbn_int(bn_17_weight, 12, 4, 1, 1)
bn_5_w_int = get_wbn_int(bn_20_weight, 12, 4, 1, 1)
bn_6_w_int = get_wbn_int(bn_23_weight, 12, 4, 1, 1)
bn_7_w_int = get_wbn_int(bn_26_weight, 12, 4, 1, 1)
print("bn_0_w_int max power of 2:", bn_0_w_int.max().item(), bn_0_w_int.min().item(), "Required bitwidth: ", math.log(power_log(bn_0_w_int.max().item()), 2))
print("bn_1_w_int max power of 2:", bn_1_w_int.max().item(), bn_1_w_int.min().item(), "Required bitwidth: ", math.log(power_log(bn_1_w_int.max().data), 2))
print("bn_2_w_int max power of 2:", bn_2_w_int.max().item(), bn_2_w_int.min().item(), "Required bitwidth: ", math.log(power_log(bn_2_w_int.max().data), 2))
print("bn_3_w_int max power of 2:", bn_3_w_int.max().item(), bn_3_w_int.min().item(), "Required bitwidth: ", math.log(power_log(bn_3_w_int.max().data), 2))
print("bn_4_w_int max power of 2:", bn_4_w_int.max().item(), bn_4_w_int.min().item(), "Required bitwidth: ", math.log(power_log(bn_4_w_int.max().item), 2))
print("bn_5_w_int max power of 2:", bn_5_w_int.max().item(), bn_5_w_int.min().item(), "Required bitwidth: ", math.log(power_log(bn_5_w_int.max().item()), 2))
print("bn_6_w_int max power of 2:", bn_6_w_int.max().item(), bn_6_w_int.min().item(), "Required bitwidth: ", math.log(power_log(bn_6_w_int.max().item()), 2))
print("bn_7_w_int max power of 2:", bn_7_w_int.max().item(), bn_7_w_int.min().item(), "Required bitwidth: ", math.log(power_log(bn_7_w_int.max().item()), 2))
bn_0_b_I = get_bbnI(bn_1_bias, 12, 4, 1, 1)
bn_0_b_int = get_bbn_int(bn_1_bias, 12, 4, 1, 1)
bn_1_b_int = get_bbn_int(bn_5_bias, 12, 4, 1, 1)
bn_2_b_int = get_bbn_int(bn_9_bias, 12, 4, 1, 1)
bn_3_b_int = get_bbn_int(bn_13_bias, 12, 4, 1, 1)
bn_4_b_int = get_bbn_int(bn_17_bias, 12, 4, 1, 1)
bn_5_b_int = get_bbn_int(bn_20_bias, 12, 4, 1, 1)
bn_6_b_int = get_bbn_int(bn_23_bias, 12, 4, 1, 1)
bn_7_b_int = get_bbn_int(bn_26_bias, 12, 4, 1, 1)
# print("bn_0_b_I: ", round(math.log(power_log(bn_0_b_I.max().item()))))
print("bn_0_b_int max power of 2: ", round(math.log(power_log(bn_0_b_int.max().item()))))
print("bn_1_b_int max power of 2: ", round(math.log(power_log(bn_1_b_int.max().item()))))
print("bn_2_b_int max power of 2: ", round(math.log(power_log(bn_2_b_int.max().item()))))
print("bn_3_b_int max power of 2: ", round(math.log(power_log(bn_3_b_int.max().item()))))
print("bn_4_b_int max power of 2: ", round(math.log(power_log(bn_4_b_int.max().item()))))
print("bn_5_b_int max power of 2: ", round(math.log(power_log(bn_5_b_int.max().item()))))
print("bn_6_b_int max power of 2: ", round(math.log(power_log(bn_6_b_int.max().item()))))
print("bn_7_b_int max power of 2: ", round(math.log(power_log(bn_7_b_int.max().item()))))

bn_0_b_int max power of 2:  21
bn_1_b_int max power of 2:  17
bn_2_b_int max power of 2:  20
bn_3_b_int max power of 2:  17
bn_4_b_int max power of 2:  18
bn_5_b_int max power of 2:  17
bn_6_b_int max power of 2:  18
bn_7_b_int max power of 2:  20


In [21]:
# bn_1_weight_int = get_w_int(bn_1_weight, 14)
# bn_5_weight_int = get_w_int(bn_1_weight, 13)
# bn_9_weight_int = get_w_int(bn_1_weight, 12)
# bn_13_weight_int = get_w_int(bn_1_weight, 11)
# bn_17_weight_int = get_w_int(bn_1_weight, 13)
# bn_20_weight_int = get_w_int(bn_1_weight, 12)
# bn_23_weight_int = get_w_int(bn_1_weight, 13)
# bn_26_weight_int = get_w_int(bn_1_weight, 14)

# bn_1_bias_int = get_w_int(bn_1_bias,26)
# bn_5_bias_int = get_w_int(bn_1_bias,21)
# bn_9_bias_int = get_w_int(bn_1_bias,22)
# bn_13_bias_int = get_w_int(bn_1_bias,21)
# bn_17_bias_int = get_w_int(bn_1_bias,21)
# bn_20_bias_int = get_w_int(bn_1_bias,21)
# bn_23_bias_int = get_w_int(bn_1_bias,22)
# bn_26_bias_int = get_w_int(bn_1_bias,23)

np.save('bn_0_weight', bn_0_w_int)
np.save('bn_0_bias', bn_0_b_int)
np.save('bn_1_weight', bn_1_w_int)
np.save('bn_1_bias', bn_1_b_int)
np.save('bn_2_weight', bn_2_w_int)
np.save('bn_2_bias', bn_2_b_int)
np.save('bn_3_weight', bn_3_w_int)
np.save('bn_3_bias', bn_3_b_int)
np.save('bn_4_weight', bn_4_w_int)
np.save('bn_4_bias', bn_4_b_int)
np.save('bn_5_weight', bn_5_w_int)
np.save('bn_5_bias', bn_5_b_int)
np.save('bn_6_weight', bn_6_w_int)
np.save('bn_6_bias', bn_6_b_int)
np.save('bn_7_weight', bn_7_w_int)
np.save('bn_7_bias', bn_7_b_int)

In [ ]:
bn_3_b = np.load('bn_3_weight.npy')
print(bn_3_b)

In [ ]:
def print_hex_line(in_f, size_value, SIMD, PE):
    print(size_value)
    print('{', end='')
    for i in range(0, size_value[0]):  # OUT_channel traverse
        if i%(size_value[0]/PE) == 0:
            print('')
            print('{', end = '')
        for k1 in range(0, size_value[2]): # kernel trverse
            for k2 in range(0, size_value[3]): # kernel traverse
                for j in range(0, size_value[1]): # In_channel traverse
                    if in_f[i][j][k1][k2] < 0:
                        in_f[i][j][k1][k2] = in_f[i][j][k1][k2] * (-1) + 8
                    if j/(size_value[1]/SIMD) == 0:
                        print('"0x', end='')
                    formatted_data = format(int(in_f[i][j][k1][k2].item()), "x")
#                     print(formatted_data, end = '')
                    print(i, end='')
                    if size_value[1] <= SIMD:
                        if i == size_value[0]/PE -1:
                            print('"}', end='')
                        else:
                            if j == size_value[1]-1:
                                print('",', end='')
#                             else:
                    else:
                        if int((j+1)%(size_value[1]/SIMD)) == 0:
                            print('",', end='')
    print('}', end = '')

In [ ]:
def weight_reorg_2d(in_f, size_value, SIMD, PE):
    print(size_value)
    K = size_value[2]
    IN_CH = size_value[1]
    OUT_CH = size_value[0]
#     weight_store[OUT_CH][OUT_CH*IN_CH*K*K]
    rows, cols = (OUT_CH, IN_CH*K*K)
    weight_store = [[0 for i in range(cols)] for j in range(rows)]
#     dim_1 = PE
#     dim_2 = K*K*OUT_CH/PE*IN_CH*SIMD
    
    for out_channel in range(0, OUT_CH):
        for in_channel in range(0, IN_CH):
            for ker1 in range(0, K):
                for ker2 in range(0, K):
                    weight_store[out_channel][in_channel*K*K + ker1*K + ker2] = int(in_f[out_channel][in_channel][ker1][ker2].item());
    return weight_store, rows, cols

def weight_print(weight_2d, rows, cols, SIMD, PE):
    print('out_dim = ', int(PE), ',', int((cols/SIMD)*(rows/PE)))
    print('{')
    for i in range(0, rows):
        if i%(rows/PE) == 0:
            print('')
            print('{')
        simd_cnt = 0
        for j in range(0, cols):
            if weight_2d[i][j] < 0:
                weight_2d[i][j] = weight_2d[i][j] * (-1) + 8
            if j%SIMD == 0:
                print('"0x', end='')
            formatted_data = format(int(weight_2d[i][j]), "x")
            print(formatted_data, end='')
            simd_cnt = simd_cnt + 1
            if simd_cnt%SIMD == 0:
                if j == cols-1 and (i+1)%(rows/PE)==0:
                    if j == cols -1 and i == rows-1:
                        print('"}')
                    else:
                        print('"},', end='')
                else:
                    print('",', end='')
    print('}')

In [ ]:
conv_0_wq = get_w_int(conv_0_weight, 4)
np.save('conv_0',conv_0_wq)
conv_1_wq = get_w_int(conv_1_weight, 4)
np.save('conv_1',conv_1_wq)
conv_2_wq = get_w_int(conv_2_weight, 4)
np.save('conv_2',conv_2_wq)
conv_3_wq = get_w_int(conv_3_weight, 4)
np.save('conv_3',conv_3_wq)
conv_4_wq = get_w_int(conv_4_weight, 4)
np.save('conv_4',conv_4_wq)
conv_5_wq = get_w_int(conv_5_weight, 4)
np.save('conv_5',conv_5_wq)
conv_6_wq = get_w_int(conv_6_weight, 4)
np.save('conv_6',conv_6_wq)
conv_7_wq = get_w_int(conv_7_weight, 4)
np.save('conv_7',conv_7_wq)
conv_8_wq = get_w_int(conv_8_weight, 4)
np.save('conv_8',conv_8_wq)
# print(conv_0_wq)

In [ ]:
a = np.load('conv_8.npy')
print(a)

In [ ]:
conv_0_wq = get_w_int(conv_8_weight, 4)
PE = 2
SIMD = 8
# print(conv_1_wq.data[0][0])
size_value = conv_0_wq.size()
# conv_0_wq = conv_0_wq.cpu()
# conv_0_wq = get_w_int(conv_0_wq, 4)
weight_store, rows, cols = weight_reorg_2d(conv_0_wq, size_value, 0, 0)
print(rows, cols)
weight_print(weight_store, rows, cols, SIMD, PE)

In [ ]:
conv_0_wq = get_w_int(conv_0_weight, 4)
PE = 2
SIMD = 8
# print(conv_1_wq.data[0][0])
size_value = conv_0_wq.size()
conv_0_wq = conv_0_wq.cpu()
print_hex_line(conv_0_wq.data, size_value, SIMD, PE)
# print(size_value[1])

In [ ]:
print(size_value)